In [36]:
import pandas as pd
from sqlalchemy import create_engine

In [37]:
patient_csv = "patient.csv"
patient_df = pd.read_csv(patient_csv)

route_csv = "route.csv"
route_df = pd.read_csv(route_csv)

time_csv = "time.csv"
time_df = pd.read_csv(time_csv)


In [40]:
rds_connection_string = "postgres:postgres@localhost:5432/ELT_Project"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [41]:
engine.table_names()

['time', 'patient', 'route']

In [42]:
patient_df.to_sql(name='patient', con=engine, if_exists='append', index=False)

In [43]:
route_df.to_sql(name='route', con=engine, if_exists='append', index=False)

In [44]:
time_df.to_sql(name='time', con=engine, if_exists='append', index=False)

In [45]:
aggregate_df = pd.read_sql_query('select A.*,B.* from patient A LEFT JOIN route B ON A.patient_id = B.patient_id', con=engine)

aggregate_df.to_csv('aggregate.csv')

aggregate_df

,patient_id,sex,birth_year,country,region,group,infection_reason,infection_order,infected_by,contact_number,...,deceased_date,state,id,patient_id,date,province,city,visit,latitude,longitude
0,1,female,1984.0,China,filtered at airport,None,visit to Wuhan,1.0,NaN,45.0,...,None,released,1.0,1.0,2020-01-19,Incheon,Jung-gu,airport,37.460459,126.440680
1,1,female,1984.0,China,filtered at airport,None,visit to Wuhan,1.0,NaN,45.0,...,None,released,2.0,1.0,2020-01-20,Incheon,Seo-gu,hospital_isolated,37.478832,126.668558
2,2,male,1964.0,Korea,filtered at airport,None,visit to Wuhan,1.0,NaN,75.0,...,None,released,3.0,2.0,2020-01-22,Gyeonggi-do,Gimpo-si,airport,37.562143,126.801884
3,2,male,1964.0,Korea,filtered at airport,None,visit to Wuhan,1.0,NaN,75.0,...,None,released,4.0,2.0,2020-01-23,Seoul,Jung-gu,hospital_isolated,37.567454,127.005627
4,3,male,1966.0,Korea,capital area,None,visit to Wuhan,1.0,NaN,16.0,...,None,released,5.0,3.0,2020-01-20,Incheon,Jung-gu,airport,37.460459,126.440680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6919,2795,None,NaN,Korea,None,None,None,NaN,NaN,NaN,...,None,isolated,NaN,NaN,None,None,None,None,NaN,NaN
6920,3328,None,NaN,Korea,None,None,None,NaN,NaN,NaN,...,None,isolated,NaN,NaN,None,None,None,None,NaN,NaN
6921,2502,None,NaN,Korea,None,None,None,NaN,NaN,NaN,...,None,isolated,NaN,NaN,None,None,None,None,NaN,NaN
6922,2824,None,NaN,Korea,None,None,None,NaN,NaN,NaN,...,None,isolated,NaN,NaN,None,None,None,None,NaN,NaN


In [46]:
duplicates = []

route_unique = []

patient_unique = []

for x in route_df.columns:
    if x in patient_df.columns:
        duplicates.append(x)

for y in route_df.columns:
    if y not in patient_df.columns:
        route_unique.append(y)

for z in patient_df.columns:
    if z not in route_df.columns:
        patient_unique.append(z)
        
duplicates = ['A.' + x + ',' for x in duplicates]

patient_unique = ['A.' + y + ',' for y in patient_unique]

route_unique = ['B.' + z + ',' for z in route_unique]

select_string = ''

for x in duplicates:
    select_string = select_string + x
    
for y in patient_unique:
    select_string = select_string + y
    
for z in route_unique:
    select_string = select_string + z

select_string = select_string[:-1]

In [47]:
routeAndPatient_df = pd.read_sql_query(f'select {select_string} from patient A LEFT JOIN route B ON A.patient_id = B.patient_id ', con=engine)

routeAndPatient_df.to_csv('routeAndPatient.csv')

In [48]:
routeAndPatient_df

,patient_id,sex,birth_year,country,region,group,infection_reason,infection_order,infected_by,contact_number,confirmed_date,released_date,deceased_date,state,date,province,city,visit,latitude,longitude
0,1,female,1984.0,China,filtered at airport,None,visit to Wuhan,1.0,NaN,45.0,2020-01-20,2020-02-06,None,released,2020-01-19,Incheon,Jung-gu,airport,37.460459,126.440680
1,1,female,1984.0,China,filtered at airport,None,visit to Wuhan,1.0,NaN,45.0,2020-01-20,2020-02-06,None,released,2020-01-20,Incheon,Seo-gu,hospital_isolated,37.478832,126.668558
2,2,male,1964.0,Korea,filtered at airport,None,visit to Wuhan,1.0,NaN,75.0,2020-01-24,2020-02-05,None,released,2020-01-22,Gyeonggi-do,Gimpo-si,airport,37.562143,126.801884
3,2,male,1964.0,Korea,filtered at airport,None,visit to Wuhan,1.0,NaN,75.0,2020-01-24,2020-02-05,None,released,2020-01-23,Seoul,Jung-gu,hospital_isolated,37.567454,127.005627
4,3,male,1966.0,Korea,capital area,None,visit to Wuhan,1.0,NaN,16.0,2020-01-26,2020-02-12,None,released,2020-01-20,Incheon,Jung-gu,airport,37.460459,126.440680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6919,2795,None,NaN,Korea,None,None,None,NaN,NaN,NaN,2020-02-29,None,None,isolated,None,None,None,None,NaN,NaN
6920,3328,None,NaN,Korea,None,None,None,NaN,NaN,NaN,2020-03-01,None,None,isolated,None,None,None,None,NaN,NaN
6921,2502,None,NaN,Korea,None,None,None,NaN,NaN,NaN,2020-02-29,None,None,isolated,None,None,None,None,NaN,NaN
6922,2824,None,NaN,Korea,None,None,None,NaN,NaN,NaN,2020-02-29,None,None,isolated,None,None,None,None,NaN,NaN
